# Importing the required libraries

In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

In [164]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
data = pd.read_csv('data.txt', sep=', ', header=None, names=columns)

# Drop the fnlwgt column
data = data.drop('fnlwgt', axis=1)

/var/folders/xs/mznlk2xj3gggmsh0bzdkjj100000gn/T/ipykernel_70111/2984815827.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('data.txt', sep=', ', header=None, names=columns)


In [165]:
# I want to count the number of ? characters in the each column
question_marks = data.isin(['?']).sum()

# Replace the ? characters with NaN
data = data.replace('?', np.nan)
data.dropna(inplace=True)

# Planning for Visualizations
## Feature Selection
We need to choose the number of features that we are continuing with (Need to be at least 8 variables)
1. Age
2. Workclass
3. Education
4. Maritial Status
5. Sex
6. Native Country
7. hours-per-week
8. Capital Gain

## User Stories
Define the 5 user stories that I want to use for the project. (At least, 3 of them must be mutli-variate)
1. Heat Map &#8594; (Native Country and Income) I CAN NOT DO THAT
    1. Useless plot
    2. The reason being that most entries are in the US
    3. I had to drop certain unsignificant columns from the dataframe
    4. All countries have had 1 or 2 entry except the US

2. Stacked Bar Chart &#8594; (Capital Gain and Income)
3. Mosaic Plot &#8594; (Sex and Income and Race possibly)
4. I could do a box and whisker plot &#8594; (HPW and salary)

In [166]:
# Split the data into the 2 dataframes one for the incomes less than and equal 50k and the other for the incomes greater than 50k
data_less_50k = data[data['income'] == '<=50K']
data_greater_50k = data[data['income'] == '>50K']